In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\Olívia\Desktop\batatinhas\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Ocorrências de inundações por Distrito
Intersecção entre a camada de distritos e a de pontos de ocorrências de inundação, pelo método 'overlay intersection'.

In [3]:
data_path=get_data_diretorio()
assets_path=os.path.join(
    data_path,
    'assets'
)

# Dependências
Este notebooks é dependente dos arquivos resultantes dos notebooks "pontos_inundacao_2024" e "../../arborizacao_viaria/malha_distritos"

In [4]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'distrito_ibge.parquet'
    )
)

In [5]:
gdf_inund = gpd.read_parquet(
    os.path.join(
        assets_path,
        'areas_inundacao',
        'pontos_inundacoes_2024.parquet'
    )
)

In [6]:
gdf_inund.shape

(237, 5)

# Overlay Intersection
Após realizar a intersecção teremos uma tabela com os pontos e as informações dos distritos em que se encontram

In [7]:
overlay_inund = gpd.overlay(
    gdf_inund,
    gdf_distritos,
    how='intersection',
    keep_geom_type=True
)

In [8]:
overlay_inund.shape

(237, 18)

# Conferir informações
É possível que algum ponto tenha sido apagado por estar na área de fronteira entre dois distritos, ou fora da área do município, devido a alguma incompatibilidade das malhas.

Primeiro, vamos ver se algum ponto não apareceu na intersecção:

In [9]:
gdf_inund.sample()

,data,ocorrencia,nm_subpref,id_ocorrencias,geometry
9,2024-02-14,INUNDACAO,SM - SAO MATEUS,32143986,POINT (355448.985 7389817.135)


In [10]:
gdf_inund.loc[~gdf_inund['id_ocorrencias'].isin(
    overlay_inund['id_ocorrencias']
)]

,data,ocorrencia,nm_subpref,id_ocorrencias,geometry


In [11]:
print(
    f'O distrito com mais ocorrências de inundações é: {
        overlay_inund['NM_DIST'].mode()[0]
    }'
)

O distrito com mais ocorrências de inundações é: ITAQUERA


# Salvar GDF

In [12]:
save_parquet_excel(
    overlay_inund,
    fname='interseccao_inundacoes2024_distritos',
    data_path=assets_path,
    data_subpath = 'areas_inundacao'
)